# Load packages

In [272]:
import pandas as pd
import numpy as np

from sklearn.metrics import cohen_kappa_score

import sqlite3
import fasttext

# First 200: Inter-rater agreement

In [273]:
r3 = pd.read_csv('../data/disease_categorization/evaluation/first_200/round 3/Crowdfunding+-+Manual+Label+101-150_December+29,+2020_08.52.csv', header=None).T.iloc[17:]
r4 = pd.read_csv('../data/disease_categorization/evaluation/first_200/round 4/Crowdfunding+-+Manual+Label+Round+4_January+7,+2021_18.13.csv', header=None).T.iloc[17:]
r5 = pd.read_csv('../data/disease_categorization/evaluation/first_200/round 5/Crowdfunding+-+Manual+Label+Round+5_January+11,+2021_08.36.csv', header=None).T.iloc[17:]
r6 = pd.read_csv('../data/disease_categorization/evaluation/first_200/round 6/Crowdfunding+-+Manual+Label+Round+6_January+15,+2021_09.35.csv', header=None).T.iloc[17:]

### Inspect data

In [274]:
print(r3.shape)
print(r4.shape)
print(r5.shape)
print(r6.shape)

(150, 5)
(150, 4)
(150, 4)
(150, 5)


In [275]:
r3.head()

,0,1,2,3,4
17,QID2,https://www.gofundme.com/f/please-help-karens-...,"Infections,Injuries and external causes,Mental...",NaN,"Infections,Mental and substance use disorders,..."
18,QID3,https://www.gofundme.com/f/please-help-karens-...,Infections,NaN,Infections
19,QID4_TEXT,https://www.gofundme.com/f/please-help-karens-...,"Allergy, unspecified, subsequent encounter-->f...",NaN,Symptoms of mental illness d/t underlying food...
20,QID5,https://www.gofundme.com/f/patrickwalter__base...,Neoplasms,NaN,Neoplasms
21,QID6,https://www.gofundme.com/f/patrickwalter__top ...,Neoplasms,NaN,Neoplasms


In [276]:
r4.head()

,0,1,2,3
17,QID2,https://www.gofundme.com/f/healthecheryl__base...,"Nervous system diseases,Other noncommunicable ...","Nervous system diseases,Other noncommunicable ..."
18,QID3,https://www.gofundme.com/f/healthecheryl__top ...,Other noncommunicable diseases,Other noncommunicable diseases
19,QID4_TEXT,https://www.gofundme.com/f/healthecheryl__NOTE...,sarcoidosis listed under non communicable dise...,MS and sarcoidosis\nmain is sarcoidosis
20,QID5,https://www.gofundme.com/f/22kcir7teo__base - ...,"Cardiovascular diseases,Genitourinary disorder...","Cardiovascular diseases,Genitourinary disorder..."
21,QID6,https://www.gofundme.com/f/22kcir7teo__top - P...,Genitourinary disorders,Infections


In [277]:
r5.head()

,0,1,2,3
17,QID2,https://www.gofundme.com/f/d90qtg__base - Hell...,Cardiovascular diseases,"Cardiovascular diseases,Nonspecific"
18,QID3,https://www.gofundme.com/f/d90qtg__top - Pleas...,Cardiovascular diseases,Cardiovascular diseases
19,QID4_TEXT,https://www.gofundme.com/f/d90qtg__NOTES - Ple...,NaN,"congestive heart failure, clots in my lungs an..."
20,QID5,https://www.gofundme.com/f/breatheeasyaimee__b...,Nonspecific,Nonspecific
21,QID6,https://www.gofundme.com/f/breatheeasyaimee__t...,Nonspecific,Nonspecific


In [278]:
r6.head()

,0,1,2,3,4
17,QID2,https://www.gofundme.com/f/nc89hu-battling-sta...,"Neoplasms,Respiratory diseases",NaN,Neoplasms
18,QID3,https://www.gofundme.com/f/nc89hu-battling-sta...,Neoplasms,NaN,Neoplasms
19,QID4_TEXT,https://www.gofundme.com/f/nc89hu-battling-sta...,acute pulmonary edema,NaN,NaN
20,QID5,https://www.gofundme.com/f/javier-montenegros-...,Genitourinary disorders,NaN,Genitourinary disorders
21,QID6,https://www.gofundme.com/f/javier-montenegros-...,Genitourinary disorders,NaN,Genitourinary disorders


### Notes from inspection

r3
- need to delete column 3, this was a blank survey response
- col2 is rater D

r4
- col2 is rater D

r5
- col2 is rater D

r6
- col2 is rater S
- col3 is rater D's second half
- col4 is rater D's first half
- need to merge 3 and 4

### Clean data

In [279]:
r3.columns = ['qid','header','rater_D','blank','rater_S']
r4.columns = ['qid', 'header', 'rater_D', 'rater_S']
r5.columns = ['qid', 'header', 'rater_D', 'rater_S']
r6.columns = ['qid', 'header', 'rater_S', 'rater_D_2', 'rater_D_1']

In [280]:
r6 = r6.reset_index()

In [281]:
r6.head(70)

,index,qid,header,rater_S,rater_D_2,rater_D_1
0,17,QID2,https://www.gofundme.com/f/nc89hu-battling-sta...,"Neoplasms,Respiratory diseases",NaN,Neoplasms
1,18,QID3,https://www.gofundme.com/f/nc89hu-battling-sta...,Neoplasms,NaN,Neoplasms
2,19,QID4_TEXT,https://www.gofundme.com/f/nc89hu-battling-sta...,acute pulmonary edema,NaN,NaN
3,20,QID5,https://www.gofundme.com/f/javier-montenegros-...,Genitourinary disorders,NaN,Genitourinary disorders
4,21,QID6,https://www.gofundme.com/f/javier-montenegros-...,Genitourinary disorders,NaN,Genitourinary disorders
...,...,...,...,...,...,...
65,82,QID67_TEXT,https://www.gofundme.com/f/lets-help-ariel-kic...,NaN,NaN,NaN
66,83,QID68,https://www.gofundme.com/f/daphnee-strong__bas...,"Neoplasms,Nonspecific",NaN,Neoplasms
67,84,QID69,https://www.gofundme.com/f/daphnee-strong__top...,Neoplasms,NaN,Neoplasms
68,85,QID70_TEXT,https://www.gofundme.com/f/daphnee-strong__NOT...,NaN,NaN,NaN


In [282]:
first = r6['rater_D_1'].iloc[:69].tolist()
second = r6['rater_D_2'].iloc[69:].tolist()
new_rater_D = first + second
r6.loc[:,'rater_D'] = new_rater_D

In [283]:
r3 = r3[['header','rater_D','rater_S']]
r4 = r4[['header','rater_D','rater_S']]
r5 = r5[['header','rater_D','rater_S']]
r6 = r6[['header','rater_D','rater_S']]

In [284]:
r3.loc[:,'round'] = [3 for x in range(len(r3))]
r4.loc[:,'round'] = [4 for x in range(len(r4))]
r5.loc[:,'round'] = [5 for x in range(len(r5))]
r6.loc[:,'round'] = [6 for x in range(len(r6))]

In [285]:
#rbind dfs
m = pd.concat([r3, r4, r5, r6])

In [286]:
#extract url from header
m['url'] = [x.split(' - ')[0].split('__')[0] for x in m['header']]

In [287]:
#extract question category from header
m['qcat'] = ['base','top','NOTES'] * 200

In [288]:
m = m[['url','qcat','rater_D','rater_S','round']]

In [289]:
#subset data

# all categories that are in text
base = m[m['qcat'] == 'base']

# top (aka primary) category
top = m[m['qcat'] == 'top']

# notes
notes = m[m['qcat'] == 'NOTES']

In [290]:
print(base.shape)
print(top.shape)
print(notes.shape)

(200, 5)
(200, 5)
(200, 5)


In [291]:
#convert data from wide to long
rater_D_melt = pd.melt(base, id_vars=['url','qcat','round'], value_vars=['rater_D'])
rater_S_melt = pd.melt(base, id_vars=['url','qcat','round'], value_vars=['rater_S'])

In [292]:
#merge melted data and rename columns
melt = pd.concat([rater_D_melt, rater_S_melt])
melt = melt.rename(columns={'variable':'rater','value':'cat'})

In [293]:
#reset index
melt = melt.reset_index()
del melt['index']

In [294]:
def comma2lofd(df):
    '''convert cat column from comma delimited to list of dictionaries with 1 or 0 for each category'''
    
    disease_categories = [
        'Cardiovascular diseases',
        'Endocrine diseases',
        'Gastrointestinal diseases',
        'Genitourinary disorders',
        'Infections',
        'Injuries and external causes',
        'Mental and substance use disorders',
        'Musculoskeletal diseases',
        'Neoplasms',
        'Nervous system diseases',
        'Respiratory diseases'
    ]
    
    r = []
    
    for index,row in df.iterrows():
        lofd = []
        as_list = row['cat'].split(',')
        for disease in disease_categories:
            if disease in as_list:
                lofd.append({disease:1})
            else:
                lofd.append({disease:0})
        r.append(lofd)
    return r

In [295]:
melt['lofd'] = comma2lofd(melt)
del melt['cat']

In [296]:
#convert to long format
long = melt.explode('lofd')

In [297]:
#extract key, value from dictionary
long.insert(loc=2, column='cat', value=[list(x.items())[0][0] for x in long['lofd']])
long.insert(loc=3, column='count', value=[list(x.items())[0][1] for x in long['lofd']])

long = long[['url','rater','round','cat','count']]

In [298]:
long['cat'] = long['cat'].replace({'Mental and substance use disorders':'Mental health disorders',
                                  'Genitourinary disorders':'Genitourinary diseases'})

In [299]:
#create pivot tables
rater_S_pivot_1 = long[long['rater'] == 'rater_S'].pivot_table(index=['url'], columns='cat', values='count', aggfunc=(pd.Series.sum), fill_value=0)
rater_D_pivot_1 = long[long['rater'] == 'rater_D'].pivot_table(index=['url'], columns='cat', values='count', aggfunc=(pd.Series.sum), fill_value=0)

In [300]:
#calculate counts and percentage by category
rater_D_counts_1 = rater_D_pivot_1.describe().T
rater_D_counts_1['N'] = [round(x) for x in rater_D_counts_1['count'] * rater_D_counts_1['mean']]
rater_D_counts_1['PER'] = rater_D_counts_1['N'] / rater_D_counts_1['count']

rater_S_counts_1 = rater_S_pivot_1.describe().T
rater_S_counts_1['N'] = [round(x) for x in rater_S_counts_1['count'] * rater_S_counts_1['mean']]
rater_S_counts_1['PER'] = rater_S_counts_1['N'] / rater_S_counts_1['count']

In [301]:
cat = []
cohens_kappa = []
for col in rater_D_pivot_1.columns:
    cat.append(col)
    cohens_kappa.append(cohen_kappa_score(rater_D_pivot_1[col], rater_S_pivot_1[col]))

kappa_df = pd.DataFrame({'category':cat, 
              'cohens kappa':cohens_kappa,
              'rater_D count':rater_D_counts_1['N'],
              'rater_S count':rater_S_counts_1['N']}).sort_values('cohens kappa', ascending=False)

In [302]:
kappa_df

,category,cohens kappa,rater_D count,rater_S count
cat,,,,
Neoplasms,Neoplasms,0.979169,81,79
Infections,Infections,0.927954,16,14
Musculoskeletal diseases,Musculoskeletal diseases,0.877900,18,18
Genitourinary diseases,Genitourinary diseases,0.871589,19,15
Endocrine diseases,Endocrine diseases,0.861624,12,11
Mental health disorders,Mental health disorders,0.825175,16,15
Injuries and external causes,Injuries and external causes,0.811083,32,23
Respiratory diseases,Respiratory diseases,0.807384,19,15
Cardiovascular diseases,Cardiovascular diseases,0.781399,31,28


# First 200: Discrepancy resolution and Reference Set

This was done on a qualitrics survey to record decisions made during a meeting to resolve discrepancies.

In [303]:
conn = sqlite3.connect("../data/db/gfm.db")
feed = pd.read_sql_query("SELECT url, fund_description FROM feed_tb", conn)

In [304]:
feed = feed.drop_duplicates('url')

### Import data from qualtrics review of discrepancies

In [305]:
resolved_top_only = pd.read_excel('../data/disease_categorization/evaluation/first_200/resolving_discrepancies/resolving_discrepancies_master.xlsx', sheet_name='top_only', header=None)
resolved_any_only = pd.read_excel('../data/disease_categorization/evaluation/first_200/resolving_discrepancies/resolving_discrepancies_master.xlsx', sheet_name='any_only', header=None)
resolved_any_and_top = pd.read_excel('../data/disease_categorization/evaluation/first_200/resolving_discrepancies/resolving_discrepancies_master.xlsx', sheet_name='any_and_top', header=None)

### Clean data

In [306]:
#rename columns
resolved_top_only.columns = ['qid','header','resp']
resolved_any_only.columns = ['qid','header','resp']
resolved_any_and_top.columns = ['qid','header','resp']

In [307]:
#extract url from header
resolved_top_only.loc[:,'url'] = [x.split(' - ')[0].split('__')[0] for x in resolved_top_only['header']]
resolved_any_only.loc[:,'url'] = [x.split(' - ')[0].split('__')[0] for x in resolved_any_only['header']]
resolved_any_and_top.loc[:,'url'] = [x.split(' - ')[0].split('__')[0] for x in resolved_any_and_top['header']]

In [308]:
print(resolved_top_only.shape)
print(resolved_any_only.shape)
print(resolved_any_and_top.shape)

(18, 4)
(132, 4)
(69, 4)


In [309]:
#add question category
resolved_top_only.loc[:,'cat'] = ['base','top','NOTES'] * int(18/3)
resolved_any_only.loc[:,'cat'] = ['base','top','NOTES'] * int(132/3)
resolved_any_and_top.loc[:,'cat'] = ['base','top','NOTES'] * int(69/3)

### Check that urls in each resolved category are unique

In [310]:
# print(len(top_only))
# print(len(any_only))
# print(len(any_and_top))

print(resolved_top_only['url'].nunique())
print(resolved_any_only['url'].nunique())
print(resolved_any_and_top['url'].nunique())

6
44
23


In [311]:
print(sum([resolved_top_only['url'].str.contains(x).any() for x in resolved_any_only['url'].unique()]))
print(sum([resolved_top_only['url'].str.contains(x).any() for x in resolved_any_and_top['url'].unique()]))
print(sum([resolved_any_only['url'].str.contains(x).any() for x in resolved_any_and_top['url'].unique()]))

0
0
0


### Build new reference

In [312]:
#define function to build new reference set
def BuildReference(m, t, a, at):
    '''
    Inputs:
        m = as above, merged responses between rater_D and rater_S
            (this will serve as the base, replace resolved discrepancies as needed)
        t = resolved_top_only
        a = resolved_any_only
        at = resolved_any_and_top
    '''
    
    #define results container
    r = []
    
    #loop through unique urls in m
    for url in m['url'].unique():
        
        #subset base data
        df = m[m['url'] == url]
        df = df.reset_index()

        if t['url'].str.contains(url).any():
            
            
            
            #susbet top data
            t_ = t[t['url'] == url]
            t_ = t_.reset_index()
            
            #get new value from resolved data
            new_value = t_[t_['url'] == url].iloc[1]['resp']
            
            #add resolved response to working base data
            df.at[1, 'rater_S'] = new_value
            
            #append results
            r.append(df)
            
        elif a['url'].str.contains(url).any():
            
            
            
            #susbet any data
            a_ = a[a['url'] == url]
            a_ = a_.reset_index()
            
            #get new value from resolved data
            new_value = a_[a_['url'] == url].iloc[0]['resp']
            
            #add resolved response to working base data
            df.at[0, 'rater_S'] = new_value
            
            #append results
            r.append(df)
            
        elif at['url'].str.contains(url).any():
            
            
            #susbet top data
            at_ = at[at['url'] == url]
            at_ = at_.reset_index()
            
            #get new value from resolved data
            new_value_any = at_[at_['url'] == url].iloc[0]['resp']
            new_value_top = at_[at_['url'] == url].iloc[1]['resp']
            
            #add resolved response to working base data
            df.at[0, 'rater_S'] = new_value_any
            df.at[1, 'rater_S'] = new_value_top
            
            #append results
            r.append(df)
            
        else:
            r.append(df)
    
    results = pd.concat(r)
    print('results shape: {0}'.format(results.shape))
    print('all done')
    return results

In [313]:
ref = BuildReference(m, resolved_top_only, resolved_any_only, resolved_any_and_top)

results shape: (600, 6)
all done


### Manually inspect some

In [314]:
resolved_top_only[:3]

,qid,header,resp,url,cat
0,QID20,https://www.gofundme.com/f/help-me-see-your-wo...,,https://www.gofundme.com/f/help-me-see-your-world,base
1,QID21,https://www.gofundme.com/f/help-me-see-your-wo...,Nervous system diseases,https://www.gofundme.com/f/help-me-see-your-world,top
2,QID22_TEXT,https://www.gofundme.com/f/help-me-see-your-wo...,interpreting primary medical condition as the ...,https://www.gofundme.com/f/help-me-see-your-world,NOTES


In [315]:
ref[ref['url'] == 'https://www.gofundme.com/f/help-me-see-your-world']

,index,url,qcat,rater_D,rater_S,round
0,35,https://www.gofundme.com/f/help-me-see-your-world,base,"Nervous system diseases,Other noncommunicable ...","Nervous system diseases,Other noncommunicable ...",3
1,36,https://www.gofundme.com/f/help-me-see-your-world,top,Other noncommunicable diseases,Nervous system diseases,3
2,37,https://www.gofundme.com/f/help-me-see-your-world,NOTES,NaN,"H47619\tCortical blindness, unspecified side o...",3


In [316]:
resolved_any_only.iloc[0]['resp']

'Infections,Injuries and external causes,Mental and substance use disorders,Nonspecific'

In [317]:
ref[ref['url'] == resolved_any_only.iloc[0]['url']].iloc[0]['rater_S']

'Infections,Injuries and external causes,Mental and substance use disorders,Nonspecific'

In [318]:
resolved_any_and_top[3:6]

,qid,header,resp,url,cat
3,QID5,https://www.gofundme.com/f/xny684__base - Disa...,"Endocrine diseases,Nonspecific",https://www.gofundme.com/f/xny684,base
4,QID6,https://www.gofundme.com/f/xny684__top - Pleas...,Endocrine diseases,https://www.gofundme.com/f/xny684,top
5,QID7_TEXT,https://www.gofundme.com/f/xny684__NOTES - Ple...,NaN,https://www.gofundme.com/f/xny684,NOTES


In [319]:
ref[ref['url'] == 'https://www.gofundme.com/f/xny684']

,index,url,qcat,rater_D,rater_S,round
0,87,https://www.gofundme.com/f/xny684,base,Endocrine diseases,"Endocrine diseases,Nonspecific",6
1,88,https://www.gofundme.com/f/xny684,top,Endocrine diseases,Endocrine diseases,6
2,89,https://www.gofundme.com/f/xny684,NOTES,NaN,NaN,6


### Clean new reference set

In [320]:
#subset columns
ref = ref[['url','qcat','rater_S']]
ref = ref.rename(columns={'rater_S':'label'})

In [321]:
#qualtrics abbreviates long urls with a suffix of ellipsis
#so check to see if any urls contain that
urls_to_replace = []
for url in ref['url'].unique().tolist():
    if '...' in url:
        urls_to_replace.append(url)
urls_to_replace

['https://www.gofundme.com/f/esperanza-para-mi-viejito-transplante-de-pulmon_...',
 'https://www.gofundme.com/f/ceiling-hoist-lift-care-system-for-tucker-thigpe...',
 'https://www.gofundme.com/f/lets-do-our-part-to-help-the-front-line-workers_...',
 'https://www.gofundme.com/f/help-noah-fight-stage-4-neuroendocrine-carcinoma...',
 'https://www.gofundme.com/f/get-mike-home-via-a-visit-to-his-mom-and-the-wat...']

In [322]:
new_urls = {}
for url in urls_to_replace:
    new_urls[url] = feed[feed['url'].str.contains(url[:-4])]['url'].tolist()[0]

In [323]:
ref = ref.replace(new_urls)

In [324]:
#checking that all were replaced
ref[~ref['url'].isin(feed['url'])]['url'].unique().tolist()

[]

In [325]:
#subset data by question category
ref_any = ref[ref['qcat'] == 'base']
ref_top = ref[ref['qcat'] == 'top']

In [326]:
print(ref_any.shape)
print(ref_top.shape)

(200, 3)
(200, 3)


In [327]:
def comma2lofd(df):
    '''convert cat column from comma delimited to list of dictionaries with 1 or 0 for each category'''
    
    disease_categories = [
        'Cardiovascular diseases',
        'Endocrine diseases',
        'Gastrointestinal diseases',
        'Genitourinary disorders',
        'Infections',
        'Injuries and external causes',
        'Mental and substance use disorders',
        'Musculoskeletal diseases',
        'Neoplasms',
        'Nervous system diseases',
        'Respiratory diseases'
    ]
    
    r = []
    
    for index,row in df.iterrows():
        lofd = []
        as_list = row['label'].split(',')
        for disease in disease_categories:
            if disease in as_list:
                lofd.append({disease:1})
            else:
                lofd.append({disease:0})
        r.append(lofd)
    return r

In [328]:
ref_any

,url,qcat,label
0,https://www.gofundme.com/f/please-help-karens-...,base,"Infections,Injuries and external causes,Mental..."
0,https://www.gofundme.com/f/patrickwalter,base,Neoplasms
0,https://www.gofundme.com/f/farid,base,Neoplasms
0,https://www.gofundme.com/f/wk4a68r,base,Neoplasms
0,https://www.gofundme.com/f/project-quotit039s-...,base,Neoplasms
...,...,...,...
0,https://www.gofundme.com/f/2tgzb38,base,"Cardiovascular diseases,Nervous system diseases"
0,https://www.gofundme.com/f/zjp9du-cancer-sucks...,base,"Injuries and external causes,Musculoskeletal d..."
0,https://www.gofundme.com/f/helpCharlieSmile,base,Musculoskeletal diseases
0,https://www.gofundme.com/f/kim-partin-couch-me...,base,Neoplasms


In [329]:
ref_any['lofd'] = comma2lofd(ref_any)

/var/folders/z5/c01kkql93wjgzp13rc24yhh00000gn/T/ipykernel_4505/1424282640.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ref_any['lofd'] = comma2lofd(ref_any)


In [330]:
#convert to long format
long = ref_any.explode('lofd')

In [331]:
#extract key, value from dictionary
long.insert(loc=2, column='cat', value=[list(x.items())[0][0] for x in long['lofd']])
long.insert(loc=3, column='count', value=[list(x.items())[0][1] for x in long['lofd']])

In [332]:
long = long[['url','cat','count']]

In [333]:
long['cat'] = long['cat'].replace({'Mental and substance use disorders':'Mental health disorders',
                                  'Genitourinary disorders':'Genitourinary diseases'})

In [334]:
#create pivot table
sd_ref = long.pivot_table(index=['url'], columns='cat', values='count', aggfunc=(pd.Series.sum), fill_value=0)

### Revisions from additional discussion

In [335]:
#Regarding https://www.gofundme.com/f/nbpzd4
#Remove GI due to vague language
sd_ref.loc['https://www.gofundme.com/f/nbpzd4','Gastrointestinal diseases'] = 0

In [336]:
#Regarding https://www.gofundme.com/f/support-for-twins-covid
#Remove GI in favor of infection
sd_ref.loc['https://www.gofundme.com/f/support-for-twins-covid','Gastrointestinal diseases'] = 0

In [337]:
#Regarding https://www.gofundme.com/f/e2km2k-your-help-will-make-a-difference
#Remove mental health due to vague language
sd_ref.loc['https://www.gofundme.com/f/e2km2k-your-help-will-make-a-difference','Mental health disorders'] = 0

In [338]:
#Regarding https://www.gofundme.com/f/KatiesCFDoubleLungTransplantFund
#Add respiratory due to mention of lung transplant that was missed
sd_ref.loc['https://www.gofundme.com/f/KatiesCFDoubleLungTransplantFund', 'Respiratory diseases'] = 1

# First 200: Additional Campaigns

Because some campaigns were eventually excluded as exclusion criteria were finalized, these additional campaigns were labelled so that the final number of campaigns after exclusion criteria would be appropriate.

In [339]:
add = pd.read_csv('../data/disease_categorization/evaluation/first_200/additional/sd_additional.csv')
add.index = add['url']
del add['url']

In [340]:
#add to first 200 IRR tabulations (because inter-rater agreement was 100% for these)
rater_S_pivot_1 = pd.concat([rater_S_pivot_1, add])
rater_D_pivot_1 = pd.concat([rater_D_pivot_1, add])

In [341]:
#add to reference set
sd_ref = pd.concat([sd_ref, add])

# Second 200: IRR

In [342]:
#import data
s = pd.read_excel('../data/disease_categorization/evaluation/second_200/1_to_200_S.xlsx', header=None)
p = pd.read_excel('../data/disease_categorization/evaluation/second_200/1_to_200_P.xlsx', header=None)
key = pd.read_csv('../data/disease_categorization/evaluation/second_200/campaigns_to_review_200.csv')

In [343]:
#clean data
s.columns = ['case','term','cat']
p.columns = ['case','term','cat']

del s['term']
del p['term']

s = s.drop_duplicates()
p = p.drop_duplicates()

In [344]:
#recode from case to url
s = s.merge(key[['name','url']], left_on='case', right_on='name')
p = p.merge(key[['name','url']], left_on='case', right_on='name')

In [345]:
s = s.replace({'infections':'Infections'})
p = p.replace({'infections':'Infections','neoplasms':'Neoplasms'})

In [346]:
#clean data
s = s[['url','cat']]
p = p[['url','cat']]

In [347]:
#combine data
s['rater'] = ['S'] * s.shape[0]
p['rater'] = ['P'] * p.shape[0]

second = pd.concat([s,p])

In [348]:
#replace na
second['cat'] = second['cat'].fillna('BLANK')

In [349]:
#create comma-delimited string
second = second.groupby(['url','rater']).agg(list).reset_index()
second['cat'] = [','.join(x) for x in second['cat']]

In [350]:
#REDEFINE FUNCTION HERE DUE TO DIFFERENT TERMINOLOGY FOR MENTAL HEALTH

def comma2lofd_v2(df):
    '''convert cat column from comma delimited to list of dictionaries with 1 or 0 for each category'''
    
    disease_categories = [
        'Cardiovascular diseases',
        'Endocrine diseases',
        'Gastrointestinal diseases',
        'Genitourinary diseases',
        'Infections',
        'Injuries and external causes',
        'Mental health disorders',
        'Musculoskeletal diseases',
        'Neoplasms',
        'Nervous system diseases',
        'Respiratory diseases'
    ]
    
    r = []
    
    for index,row in df.iterrows():
        lofd = []
        as_list = row['cat'].split(',')
        for disease in disease_categories:
            if disease in as_list:
                lofd.append({disease:1})
            else:
                lofd.append({disease:0})
        r.append(lofd)
    return r

In [351]:
second['lofd'] = comma2lofd_v2(second)
del second['cat']

In [352]:
#convert to long format
long_2 = second.explode('lofd')

In [353]:
#extract key, value from dictionary
long_2.insert(loc=1, column='cat', value=[list(x.items())[0][0] for x in long_2['lofd']])
long_2.insert(loc=2, column='count', value=[list(x.items())[0][1] for x in long_2['lofd']])

long_2 = long_2[['url','rater','cat','count']]

In [354]:
#create pivot tables
rater_S_pivot_2 = long_2[long_2['rater'] == 'S'].pivot_table(index=['url'], columns='cat', values='count', aggfunc=(pd.Series.sum), fill_value=0)
rater_P_pivot_1 = long_2[long_2['rater'] == 'P'].pivot_table(index=['url'], columns='cat', values='count', aggfunc=(pd.Series.sum), fill_value=0)

In [355]:
#calculate counts and percentage by category
rater_P_counts_1 = rater_P_pivot_1.describe().T
rater_P_counts_1['N'] = [round(x) for x in rater_P_counts_1['count'] * rater_P_counts_1['mean']]
rater_P_counts_1['PER'] = rater_P_counts_1['N'] / rater_P_counts_1['count']

rater_S_counts_2 = rater_S_pivot_2.describe().T
rater_S_counts_2['N'] = [round(x) for x in rater_S_counts_2['count'] * rater_S_counts_2['mean']]
rater_S_counts_2['PER'] = rater_S_counts_2['N'] / rater_S_counts_2['count']

In [356]:
cat = []
cohens_kappa = []
for col in rater_P_pivot_1.columns:
    cat.append(col)
    cohens_kappa.append(cohen_kappa_score(rater_P_pivot_1[col], rater_S_pivot_2[col]))

kappa_df = pd.DataFrame({'category':cat, 
              'cohens kappa':cohens_kappa,
              'rater_P count':rater_P_counts_1['N'],
              'rater_S count':rater_S_counts_2['N']}).sort_values('cohens kappa', ascending=False)

In [357]:
kappa_df

,category,cohens kappa,rater_P count,rater_S count
cat,,,,
Endocrine diseases,Endocrine diseases,1.000000,7,7
Injuries and external causes,Injuries and external causes,0.974962,22,23
Neoplasms,Neoplasms,0.948959,83,88
Cardiovascular diseases,Cardiovascular diseases,0.945205,50,46
Infections,Infections,0.927928,15,15
Respiratory diseases,Respiratory diseases,0.903800,11,11
Mental health disorders,Mental health disorders,0.886364,5,4
Nervous system diseases,Nervous system diseases,0.823165,26,26
Genitourinary diseases,Genitourinary diseases,0.822774,13,11


# Second 200: Discrepancy Resolution and Reference Set

This was manually created based on discussion to resolve discrepancies.

In [358]:
#import
sp_ref = pd.read_excel('../data/disease_categorization/evaluation/second_200/sp_reference.xlsx', header=None)

In [359]:
#clean data
sp_ref.columns = ['case','term','cat']

del sp_ref['term']

sp_ref = sp_ref.drop_duplicates()

In [360]:
#recode from case to url
sp_ref = sp_ref.merge(key[['name','url']], left_on='case', right_on='name')

In [361]:
sp_ref = sp_ref.replace({'infections':'Infections'})

In [362]:
#clean data
sp_ref = sp_ref[['url','cat']]

In [363]:
#replace na
sp_ref['cat'] = sp_ref['cat'].fillna('BLANK')

In [364]:
#create comma-delimited string
sp_ref = sp_ref.groupby(['url']).agg(list).reset_index()
sp_ref['cat'] = [','.join(x) for x in sp_ref['cat']]

In [365]:
sp_ref['lofd'] = comma2lofd_v2(sp_ref)
del sp_ref['cat']

In [366]:
#convert to long format
long_3 = sp_ref.explode('lofd')

In [367]:
#extract key, value from dictionary
long_3.insert(loc=0, column='cat', value=[list(x.items())[0][0] for x in long_3['lofd']])
long_3.insert(loc=1, column='count', value=[list(x.items())[0][1] for x in long_3['lofd']])

long_3 = long_3[['url','cat','count']]

In [368]:
#create pivot table
sp_ref = long_3.pivot_table(index=['url'], columns='cat', values='count', aggfunc=(pd.Series.sum), fill_value=0)

# Apply exclusion criteria from master data

In [369]:
#import master data
master = pd.read_sql_query("SELECT url, fund_description FROM master", conn)

In [370]:
#combine IRR data
rat1_pivot = pd.concat([rater_S_pivot_1, rater_S_pivot_2])
rat2_pivot = pd.concat([rater_D_pivot_1, rater_P_pivot_1])

#replace urls with ellipsis suffix as above
rat1_pivot = rat1_pivot.rename(index=new_urls)
rat2_pivot = rat2_pivot.rename(index=new_urls)


In [371]:
#combine reference set data
reference_set = pd.concat([sd_ref, sp_ref])

In [372]:
#apply exclusions
rat1_pivot = rat1_pivot.loc[rat1_pivot.index.isin(master['url'])]
rat2_pivot = rat2_pivot.loc[rat2_pivot.index.isin(master['url'])]
reference_set = reference_set.loc[reference_set.index.isin(master['url'])]

In [373]:
print(rat1_pivot.shape)
print(rat2_pivot.shape)
print(reference_set.shape)

(402, 11)
(402, 11)
(402, 11)


In [374]:
#get two random urls to delete
first = 'https://www.gofundme.com/f/chris039s-surgery-costs'
second = 'https://www.gofundme.com/f/masks-for-nurses-caregivers-front-line'

In [375]:
rat1_pivot = rat1_pivot.drop([first,second])
rat2_pivot = rat2_pivot.drop([first,second])
reference_set = reference_set.drop([first,second])

In [376]:
print(rat1_pivot.shape)
print(rat2_pivot.shape)
print(reference_set.shape)

(400, 11)
(400, 11)
(400, 11)


# Final Cohen's Kappa

In [377]:
#rater 1 data
rat1_counts = rat1_pivot.describe().T
rat1_counts['N'] = [round(x) for x in rat1_counts['count'] * rat1_counts['mean']]
rat1_counts['PER'] = rat1_counts['N'] / rat1_counts['count']

In [378]:
#rater 2 data
rat2_counts = rat2_pivot.describe().T
rat2_counts['N'] = [round(x) for x in rat2_counts['count'] * rat2_counts['mean']]
rat2_counts['PER'] = rat2_counts['N'] / rat2_counts['count']

In [379]:
cat = []
cohens_kappa = []
for col in rat2_pivot.columns:
    cat.append(col)
    cohens_kappa.append(cohen_kappa_score(rat2_pivot[col], rat1_pivot[col]))

kappa_df = pd.DataFrame({'category':cat, 
              'cohens kappa':cohens_kappa,
              'rater 2 count':rat2_counts['N'],
              'rater 1 count':rat1_counts['N']}).sort_values('cohens kappa', ascending=False)

In [380]:
kappa_df

,category,cohens kappa,rater 2 count,rater 1 count
Neoplasms,Neoplasms,0.963796,162,165
Infections,Infections,0.925651,30,28
Endocrine diseases,Endocrine diseases,0.914990,19,18
Injuries and external causes,Injuries and external causes,0.887735,55,47
Cardiovascular diseases,Cardiovascular diseases,0.882316,83,76
Genitourinary diseases,Genitourinary diseases,0.851412,32,26
Musculoskeletal diseases,Musculoskeletal diseases,0.846792,45,43
Respiratory diseases,Respiratory diseases,0.846449,30,26
Mental health disorders,Mental health disorders,0.842126,21,19
Nervous system diseases,Nervous system diseases,0.758065,60,56


# Export reference set

In [381]:
reference_set.to_csv('../data/disease_categorization/evaluation/final_reference.csv')